# Dog VS Cat - Kaggle Competition

- 进入Top20 要求 Loss < 0.04183
- 进入Top10 要求 Loss < 0.03807

In [5]:
import os
import sys
import cv2
import h5py
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from datetime import datetime
from tqdm import tqdm
from utils import get_params_count

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.applications import inception_v3, xception, resnet50, vgg16, vgg19
from keras.applications import InceptionV3, Xception, ResNet50, VGG16, VGG19
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Lambda
from keras.layers.noise import GaussianDropout
from keras.layers.pooling import GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.optimizers import SGD

In [2]:
height = 299
labels = np.array([0] * 12500 + [1] * 12500)
train = np.zeros((25000, height, height, 3), dtype=np.uint8)
test = np.zeros((12500, height, height, 3), dtype=np.uint8)

for i in tqdm(range(12500)):
    img = cv2.imread('./train/cat/%s.jpg' % str(i))
    img = cv2.resize(img, (height, height))
    train[i] = img[:, :, ::-1]
    
for i in tqdm(range(12500)):
    img = cv2.imread('./train/dog/%s.jpg' % str(i))
    img = cv2.resize(img, (height, height))
    train[i + 12500] = img[:, :, ::-1]

for i in tqdm(range(12500)):
    img = cv2.imread('./test/%s.jpg' % str(i + 1))
    img = cv2.resize(img, (height, height))
    test[i] = img[:, :, ::-1]
    
print('Training Data Size = %.2f GB' % (sys.getsizeof(train)/1024**3))
print('Testing Data Size = %.2f GB' % (sys.getsizeof(test)/1024**3))

100%|█████████████████████████████████████████████████████| 12500/12500 [00:26<00:00, 469.68it/s]


Training Data Size = 6.24 GB
Testing Data Size = 3.12 GB


## 0. CONV + GAP + Dropout + Classifier

In [31]:
X_train, X_val, y_train, y_val = train_test_split(train, labels, shuffle=True, test_size=0.2, random_state=42)

In [30]:
def finetune(MODEL, preprocess, height, freeze_till, lr, nb_epoch, weights=None):
    # Preprocess: Standardization
    x = Input(shape=(height, height, 3))
    x = Lambda(preprocess)(x)

    # Base Model: Freeze all conv layers
    base_model = MODEL(include_top=False, input_tensor=x, weights='imagenet', pooling='avg')
    for layer in base_model.layers:
        layer.trainable = True
    for layer in base_model.layers[:freeze_till]:
        layer.trainable = False

    # Customized Classifier
    y = Dropout(0.2)(base_model.output)
    y = Dense(1, activation='sigmoid', kernel_initializer='he_normal')(y)

    # Full Model: Pre-train Conv + Customized Classifier
    model = Model(inputs=base_model.input, outputs=y, name='Transfer_Learning')
    sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print('Trainable: %d, Non-Trainable: %d' % get_params_count(model))
    
    if weights is not None:
        model.load_weights(weights)
    
    # Prepare Callbacks for Model Checkpoint, Early Stopping and Tensorboard.
    log_name = '/DogVSCat-EP{epoch:02d}-LOSS{val_loss:.4f}.h5'
    log_dir = datetime.now().strftime('transfer_model_%Y%m%d_%H%M')
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)

    es = EarlyStopping(monitor='val_loss', patience=20)
    mc = ModelCheckpoint(log_dir + log_name, monitor='val_loss', save_best_only=True)
    tb = TensorBoard(log_dir=log_dir)

    model.fit(x=X_train, y=y_train, batch_size=16, epochs=nb_epoch, 
              validation_data=(X_val, y_val), callbacks=[es, mc, tb])

### 锁定前116层

In [32]:
finetune(Xception, xception.preprocess_input, height, freeze_till=116, lr=0.045, nb_epoch=1)

Trainable: 6790433, Non-Trainable: 14073096
Train on 20000 samples, validate on 5000 samples
Epoch 1/1
20000/20000 [==============================] - 226s - loss: 0.0407 - acc: 0.9853 - val_loss: 0.0232 - val_acc: 0.9908


### 锁定前86层

In [34]:
log = './transfer_model_20171021_1548/DogVSCat-EP00-LOSS0.0232.h5'
finetune(Xception, xception.preprocess_input, height, freeze_till=86, lr=0.001, nb_epoch=1, weights=log)

Trainable: 11632361, Non-Trainable: 9231168
Train on 20000 samples, validate on 5000 samples
Epoch 1/1
20000/20000 [==============================] - 288s - loss: 0.0157 - acc: 0.9951 - val_loss: 0.0193 - val_acc: 0.9930


### 锁定前57层

In [38]:
log = './transfer_model_20171021_1601/DogVSCat-EP00-LOSS0.0193.h5'
finetune(Xception, xception.preprocess_input, height, freeze_till=57, lr=0.0001, nb_epoch=1, weights=log)

Trainable: 19702241, Non-Trainable: 1161288
Train on 20000 samples, validate on 5000 samples
Epoch 1/1
20000/20000 [==============================] - 386s - loss: 0.0127 - acc: 0.9963 - val_loss: 0.0190 - val_acc: 0.9932


## 1. CONV + Dropout + GAP + Dense1024 + Dropout + Classifier

In [3]:
X_train, X_val, y_train, y_val = train_test_split(train, labels, shuffle=True, test_size=0.2, random_state=42)

In [41]:
def finetune_dense(MODEL, preprocess, height, freeze_till, lr, nb_epoch, weights=None):
    # Preprocess: Standardization
    x = Input(shape=(height, height, 3))
    x = Lambda(preprocess)(x)

    # Base Model: Freeze all conv layers
    base_model = MODEL(include_top=False, input_tensor=x, weights='imagenet')
    for layer in base_model.layers:
        layer.trainable = True
    for layer in base_model.layers[:freeze_till]:
        layer.trainable = False

    # Customized Classifier
    y = GaussianDropout(0.2)(base_model.output)
    y = GlobalAveragePooling2D()(y)
    y = Dense(1024, activation='selu', kernel_initializer='he_normal')(y)
    y = GaussianDropout(0.2)(y)
    y = Dense(1, activation='sigmoid', kernel_initializer='he_normal')(y)

    # Full Model: Pre-train Conv + Customized Classifier
    model = Model(inputs=base_model.input, outputs=y, name='Transfer_Learning')
    sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print('Trainable: %d, Non-Trainable: %d' % get_params_count(model))
    
    if weights is not None:
        model.load_weights(weights)
    
    # Prepare Callbacks for Model Checkpoint, Early Stopping and Tensorboard.
    log_name = '/DogVSCat-EP{epoch:02d}-LOSS{val_loss:.4f}.h5'
    log_dir = datetime.now().strftime('transfer_model_%Y%m%d_%H%M')
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)

    es = EarlyStopping(monitor='val_loss', patience=20)
    mc = ModelCheckpoint(log_dir + log_name, monitor='val_loss', save_best_only=True)
    tb = TensorBoard(log_dir=log_dir)

    model.fit(x=X_train, y=y_train, batch_size=16, epochs=nb_epoch, 
              validation_data=(X_val, y_val), callbacks=[es, mc, tb])

### 锁定前116层

In [42]:
finetune_dense(Xception, xception.preprocess_input, height, freeze_till=116, lr=0.045, nb_epoch=1)

Trainable: 8887585, Non-Trainable: 14073096
Train on 20000 samples, validate on 5000 samples
Epoch 1/1
20000/20000 [==============================] - 234s - loss: 0.0485 - acc: 0.9833 - val_loss: 0.0327 - val_acc: 0.9880


### 锁定前86层

In [43]:
log = './transfer_model_20171021_1622/DogVSCat-EP00-LOSS0.0327.h5'
finetune_dense(Xception, xception.preprocess_input, height, freeze_till=86, lr=0.001, nb_epoch=1, weights=log)

Trainable: 13729513, Non-Trainable: 9231168
Train on 20000 samples, validate on 5000 samples
Epoch 1/1
20000/20000 [==============================] - 293s - loss: 0.0172 - acc: 0.9949 - val_loss: 0.0229 - val_acc: 0.9916


### 锁定前57层

In [44]:
log = './transfer_model_20171021_1628/DogVSCat-EP00-LOSS0.0229.h5'
finetune_dense(Xception, xception.preprocess_input, height, freeze_till=57, lr=0.0001, nb_epoch=1, weights=log)

Trainable: 18571441, Non-Trainable: 4389240
Train on 20000 samples, validate on 5000 samples
Epoch 1/1
20000/20000 [==============================] - 352s - loss: 0.0110 - acc: 0.9966 - val_loss: 0.0225 - val_acc: 0.9920


## Solution 2: Extract Feature Vector

In [6]:
def export_gap(MODEL, preprocess=None, batch_size=128):
    x = Input(shape=(height, height, 3))
    x = Lambda(preprocess)(x)
    model = MODEL(include_top=False, input_tensor=x, weights='imagenet', pooling='avg')
    train_gap = model.predict(train, batch_size=batch_size)
    test_gap = model.predict(test, batch_size=batch_size)
    with h5py.File("gap_%s.h5" % MODEL.__name__, 'w') as f:
        f.create_dataset('train', data=train_gap)
        f.create_dataset('test', data=test_gap)

In [4]:
export_gap(InceptionV3, inception_v3.preprocess_input)

In [7]:
export_gap(Xception, xception.preprocess_input, 64)

In [19]:
train_temp = []
test_temp = []
# 'gap_InceptionV3.h5', 'gap_Xception.h5'
for gapfile in ['gap_Xception.h5']:
    with h5py.File(gapfile, 'r') as f:
        train_temp.append(np.array(f['train']))
        test_temp.append(np.array(f['test']))
train_gap = np.concatenate(train_temp, axis=1)
test_gap = np.concatenate(test_temp, axis=1)

In [20]:
X_train, X_val, y_train, y_val = train_test_split(train_gap, labels, shuffle=True, test_size=0.2, random_state=42)

In [21]:
x = Input(shape=(X_train.shape[1],))
y = Dropout(0.2)(x)
y = Dense(1, activation='sigmoid', kernel_initializer='he_normal', name='classifier')(y)
model_gap = Model(inputs=x, outputs=y, name='GAP')
model_gap.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
print('Trainable: %d, Non-Trainable: %d' % get_params_count(model_gap))

Trainable: 2049, Non-Trainable: 0


In [22]:
# Prepare Callbacks for Model Checkpoint, Early Stopping and Tensorboard.
log_name = '/DogVSCat-EP{epoch:02d}-LOSS{val_loss:.4f}.h5'
log_dir = datetime.now().strftime('gap_model_%Y%m%d_%H%M')
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

es = EarlyStopping(monitor='val_loss', patience=20)
mc = ModelCheckpoint(log_dir + log_name, monitor='val_loss', save_best_only=True)
tb = TensorBoard(log_dir=log_dir)

model_gap.fit(x=X_train, y=y_train, batch_size=16, epochs=5, validation_data=(X_val, y_val), callbacks=[es, mc, tb])

# Use all training data.
# model_gap.fit(x=train_gap, y=labels, batch_size=16, shuffle=True, epochs=5)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 4s - loss: 0.0870 - acc: 0.9871 - val_loss: 0.0326 - val_acc: 0.9910
Epoch 2/5
20000/20000 [==============================] - 3s - loss: 0.0255 - acc: 0.9934 - val_loss: 0.0265 - val_acc: 0.9922
Epoch 3/5
20000/20000 [==============================] - 4s - loss: 0.0215 - acc: 0.9939 - val_loss: 0.0244 - val_acc: 0.9922
Epoch 4/5
20000/20000 [==============================] - 3s - loss: 0.0196 - acc: 0.9940 - val_loss: 0.0236 - val_acc: 0.9922
Epoch 5/5
20000/20000 [==============================] - 3s - loss: 0.0186 - acc: 0.9943 - val_loss: 0.0232 - val_acc: 0.9920


In [37]:
y_pred = model_gap.predict(test_gap)
y_pred = y_pred.clip(min=0.005, max=0.995)

In [38]:
with open('test.csv', 'w') as f:
    f.writelines('id,label\n')
    for i in range(12500):
        f.writelines(str(i+1) + ',' + str(y_pred[i][0]) + '\n')

## 结论

- 正确的使用Finetune，的确可以达到比特征向量法更好的效果（0.0190 vs 0.0232）。关键在于掌控好学习率随解锁层数的动态变化。